PREPROCESAMIENTO

In [40]:
# 1. LIBRERÍAS
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

In [ ]:
# 2. Definir rutas
raw_paths = {
    "antivirus_etiquetado":r"C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\Logs-antivirus_etiquetado.csv",
    "windows_estaciones": r"C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\ML_event_windows_estaciones.csv",
    "windows_server": r"C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\ML_event_windows_server_correo_interno.csv",
    "waf": r"C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\ML_WAF.csv",
    "antimalware": r"C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\ML_antimalware.csv",
    "antivirus_kaspersky": r"C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\ML_antivirus_kaspersky.csv"
}

df_list = []
for name, path in raw_paths.items():
    if not os.path.exists(path):
        print(f"Archivo no encontrado: {path}")
    else:
        print(f"Cargando {path}...")
        df = pd.read_csv(path)
        df['source'] = name
        df_list.append(df)

# Unir todos los logs con su identificador de fuente
if not df_list:
    raise ValueError("No se cargó ningún archivo. Revisa las rutas.")
df_total = pd.concat(df_list, ignore_index=True)

Cargando C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\Logs-antivirus_etiquetado.csv...
Cargando C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\ML_event_windows_estaciones.csv...
Cargando C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\ML_event_windows_server_correo_interno.csv...
Cargando C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\ML_WAF.csv...
Cargando C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\ML_antimalware.csv...
Cargando C:\Users\alons\Desktop\Tesis_SIEM_ML\data\raw\ML_antivirus_kaspersky.csv...


In [42]:
# 3. Convertir timestamp
df_total['@timestamp'] = pd.to_datetime(df_total['@timestamp'], format="%b %d, %Y @ %H:%M:%S.%f", errors='coerce')
df_total = df_total.dropna(subset=['@timestamp'])  # eliminamos las fechas inválidas

df_total['hour'] = df_total['@timestamp'].dt.hour
df_total['dayofweek'] = df_total['@timestamp'].dt.dayofweek
df_total['month'] = df_total['@timestamp'].dt.month
df_total['year'] = df_total['@timestamp'].dt.year

# 4. Seleccionar columnas clave (puedes ajustar según el análisis)
cols_to_keep = ['@timestamp', 'host.ip', 'rule.name', 'rule.category', 'rule.description',
                'file.path', 'file.hash.md5', 'file.hash.sha256', 'virus.name',
                'danger.type', 'source', 'hour', 'dayofweek', 'month', 'year']

# No reemplazar fechas por "desconocido"
cols_excluding_timestamp = [c for c in cols_to_keep if c != '@timestamp']
df_clean = df_total[cols_to_keep].copy()
df_clean[cols_excluding_timestamp] = df_clean[cols_excluding_timestamp].fillna("desconocido")
print(df_total['@timestamp'].dropna().head(10))


0   2024-09-29 12:06:47
1   2024-09-27 12:06:15
2   2024-09-27 07:56:15
3   2024-09-27 07:56:15
4   2024-09-26 16:52:33
5   2024-09-26 13:00:48
6   2024-09-25 16:03:45
7   2024-09-25 15:59:35
8   2024-09-25 14:36:30
9   2024-09-25 14:36:30
Name: @timestamp, dtype: datetime64[ns]


In [43]:
# 5. Codificación categórica
cat_cols = ['host.ip', 'rule.name', 'rule.category', 'rule.description',
            'file.path', 'file.hash.md5', 'file.hash.sha256',
            'virus.name', 'danger.type', 'source']

encoders = {}
for col in cat_cols:
    enc = LabelEncoder()
    df_clean[col] = enc.fit_transform(df_clean[col])
    encoders[col] = enc

In [44]:
# 6. Guardar datos procesados
df_clean.to_csv(r"C:\Users\alons\Desktop\Tesis_SIEM_ML\data\processed\logs_unificados_limpios.csv", index=False)
print(df.head())

                   @timestamp       host.ip    host.hostname  \
0  Sep 7, 2024 @ 10:33:15.000    10.3.28.71  10443-077700362   
1  Sep 7, 2024 @ 10:32:11.000  10.4.248.188  10251-002900097   
2  Sep 7, 2024 @ 10:27:00.000  10.4.236.127  60041-071685072   
3  Sep 7, 2024 @ 10:25:13.000    10.5.28.37  93245-007700224   
4  Sep 7, 2024 @ 10:25:13.000    10.5.28.37  93245-007700224   

                         rule.name            rule.category  \
0  GNRL_EV_DEVCTRL_DEV_PLUG_DENIED  Control de dispositivos   
1  GNRL_EV_DEVCTRL_DEV_PLUG_DENIED  Control de dispositivos   
2  GNRL_EV_DEVCTRL_DEV_PLUG_DENIED  Control de dispositivos   
3  GNRL_EV_DEVCTRL_DEV_PLUG_DENIED           Device Control   
4  GNRL_EV_DEVCTRL_DEV_PLUG_DENIED           Device Control   

                         rule.description                file.path  \
0  Operación con el dispositivo prohibida    \Device\Harddisk1\DR1   
1  Operación con el dispositivo prohibida  \Device\HarddiskVolume5   
2  Operación con el dispos